# BioBERT Dual Lifestyle Classifier

This notebook demonstrates the BioBERT classification pipeline for distinguishing between dual and solo lifestyles in biological abstracts.

## 1. Data Loading and Preprocessing

Load the training and test datasets, and show the train/test split.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Display basic info
print('Training set shape:', train_df.shape)
print('Test set shape:', test_df.shape)
print('\nTraining set class distribution:')
print(train_df['label'].value_counts())
print('\nTest set class distribution:')
print(test_df['label'].value_counts())

# Show sample data
print('\nSample training data:')
print(train_df.head())
print('\nSample test data:')
print(test_df.head())

: 

## 2. Model Fine-tuning with Training Curves

Load the fine-tuned BioBERT model and display training curves.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
import json
import matplotlib.pyplot as plt

# Load the fine-tuned model and tokenizer
model_path = 'fine_tuned_biobert_classification'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load training history from trainer state
trainer_state_path = 'results/trainer_state.json'  # Using the latest checkpoint
with open(trainer_state_path, 'r') as f:
    trainer_state = json.load(f)

# Extract training metrics
log_history = trainer_state['log_history']

# Plot training curves
steps = [entry.get('step', 0) for entry in log_history if 'loss' in entry]
train_loss = [entry['loss'] for entry in log_history if 'loss' in entry]
eval_loss = [entry.get('eval_loss', None) for entry in log_history if 'eval_loss' in entry]
eval_accuracy = [entry.get('eval_accuracy', None) for entry in log_history if 'eval_accuracy' in entry]

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(steps, train_loss, label='Training Loss')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(1, 3, 2)
valid_steps = [entry.get('step', 0) for entry in log_history if 'eval_loss' in entry]
plt.plot(valid_steps, eval_loss, label='Validation Loss')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Validation Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(valid_steps, eval_accuracy, label='Validation Accuracy')
plt.xlabel('Step')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.legend()

plt.tight_layout()


## 3. Evaluation on Test Set

Evaluate the model on the test set and compute metrics: accuracy, precision, recall, F1.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import torch
from torch.utils.data import Dataset

# Custom dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare test dataset
test_texts = test_df['abstract'].tolist()
test_labels = test_df['label'].tolist()
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

# Create trainer for evaluation
trainer = Trainer(model=model)

# Get predictions
predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()
true_labels = test_labels

# Calculate metrics
accuracy = accuracy_score(true_labels, preds)
precision = precision_score(true_labels, preds, average='binary')
recall = recall_score(true_labels, preds, average='binary')
f1 = f1_score(true_labels, preds, average='binary')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('\nClassification Report:')
print(classification_report(true_labels, preds, target_names=['Solo', 'Dual']))

## 4. Example Predictions

Make predictions on example abstracts.

In [ ]:
# Example abstracts
examples = [
    "This fungus can switch between saprotrophic decomposition of dead organic matter and forming mutualistic symbioses with plant roots, demonstrating a dual trophic lifestyle.",
    "This saprotrophic fungus specializes exclusively in decomposing dead plant material and cannot form symbiotic associations.",
    "The pathogenic fungus infects living plants but can also survive saprotrophically on dead plant tissues, exhibiting dual trophic modes.",
    "This obligate symbiont fungus can only survive in mutualistic association with its host plant and cannot decompose dead organic matter independently."
]

# Tokenize and predict
model.eval()
predictions = []
for text in examples:
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=1).item()
        predictions.append('Dual' if pred == 1 else 'Solo')

# Display results
for i, (text, pred) in enumerate(zip(examples, predictions)):
    print(f'Example {i+1}: {pred}')
    print(f'Text: {text}')
